In [2]:
# VARIATIONAL AUTO-ENCODER
# Fits a VAE to the data and save the fitted functions
# Same as vae_M1 but
# This version of vae estimates the residual variance of the observations.

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from pathlib import Path
import scipy.stats as scs
import argparse



In [3]:
# ## Create the sampling layer
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class Encoder(layers.Layer):
    """Encode the data into the latent dimensional space."""

    def call(self, inputs):
        x = layers.Flatten()(inputs)
        x = tf.keras.layers.Dropout(.3)(x)
        x = layers.Dense(100, activation="tanh")(x)

        z_mean = layers.Dense(100, activation="tanh")(x)
        z_mean = layers.Dense(50, activation="tanh")(x)
        z_mean = layers.Dense(latent_dim, name="z_mean", activation="linear")(z_mean)

        z_log_var = layers.Dense(100, activation="tanh")(x)
        z_log_var = layers.Dense(50, activation="tanh")(z_log_var)
        z_log_var = layers.Dense(latent_dim, name="z_log_var", activation="linear")(z_log_var)
        z = Sampling()([z_mean, z_log_var])
        return [z_mean, z_log_var, z]


class Decoder(layers.Layer):
    """Decode the data with the structure identifying the functions."""

    def call(self, inputs):
        # ## Create the Decoder network
        # For each of the latent variables, we create a NN. The last layer takes their sum.
        x_output_list = []

        # now split
        latent_split = layers.Lambda(lambda x: tf.split(x, x.shape[1], axis=1))(inputs)
        for i in range(inputs.shape[-1]):
            x = layers.Dense(100, activation="tanh")(latent_split[i])

            x_output = layers.Dense(100, activation="tanh")(x)
            x_output = layers.Dense(50, activation="tanh")(x_output)
            x_output = layers.Dense(K, activation="tanh")(x_output)
            x_output = layers.Dense(inputs.shape[1], activation="linear")(x_output)
            x_output = layers.Reshape(inputs.shape)(x_output)
            x_output_list.append(x_output)

        x_output = layers.Add(name="list_of_g")(x_output_list)

        return x_output


# ## Create the variational autoencoder
class VAE(keras.Model):
    def __init__(self, encoder, decoder, K, known_variance, **kwargs):
        super().__init__(**kwargs)
        self.K = K
        self.known_variance=known_variance
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = 0.5 * tf.reduce_mean(tf.reduce_sum((data - reconstruction)**2 / self.known_variance + tf.math.log(self.known_variance) + tf.math.log(2. * np.pi), axis=1))
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "total_loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


def generate_vae(input_shape, latent_dim, K=5, known_variance=1.):
    # ## Create the encoder network
    """ Create the encoder network; K is the dimension of the last layer. A small number helps not to overfit. This is determined by n, see the paper for references."""

    encoder_inputs = keras.Input(shape=input_shape)
    z_mean, z_logvar, z

    vae = VAE(encoder, decoder, K=K, known_variance=known_variance)

    return(vae)


In [4]:
def preprocessing(path):
    # we pre-train the vae by randomizing the column orders
    data = pd.read_csv(path)
    data = np.array(data, dtype='float32')
    data = data[:,np.random.choice(data.shape[1], data.shape[1], replace=False)]
    data = np.expand_dims(data, axis=-1)
    return data


In [5]:
def vae_fit(data, K=5, known_variance=1., latent_dim=5):
    callback = tf.keras.callbacks.EarlyStopping(monitor='total_loss', patience=50)
    vae = generate_vae(input_shape=data.shape[1:], latent_dim=latent_dim, K=K, known_variance=known_variance)
    vae.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
    vae.fit(data, epochs=1000, callbacks=[callback])

    # We now create a model to give as output the `latent_dim` functions evaluated for a given $z$.

    intermediate_layer_model = keras.Model(inputs = vae.decoder.input,
                                        outputs = vae.decoder.get_layer("list_of_g").input)

    return(vae, intermediate_layer_model)



## Get the estimated decoder functions

In [11]:
def save_output(data, vae, intermediate_model, file, PATH_TO_FIT):

    # save g
    dist = scs.norm(0,1)
    prob = np.arange(1,201)/201.
    z = dist.ppf(prob)
    z = np.repeat(z[:, np.newaxis], num_latent, axis=1)

    y_list = intermediate_model.predict(z)

    for k in range(len(y_list)):
        y = y_list[k].squeeze()
        zy = np.concatenate((z[:,0, np.newaxis], y), axis=1)
        df = pd.DataFrame(
            data=zy, 
            columns=["z"] + ["g"+str(i+1) for i in range(y.shape[1])])

        # create folder
        outdir = PATH_TO_FIT/f"g{(k+1)}"
        outdir.mkdir(parents=True, exist_ok=True)

        # save
        df.to_csv(outdir/file, index=False)

    # save z

    z, _, _ = vae.encoder(data)
    df = pd.DataFrame(
        data = z,
        columns = ["z" + str(i+1) for i in range(z.shape[1])]
    )
    # create folder
    outdir = PATH_TO_FIT/"z"
    outdir.mkdir(parents=True, exist_ok=True)

    # save
    df.to_csv(outdir/file, index=False)
    

In [15]:
num_latent = 5

PATH_TO_DATA = Path("data/q5/x")
PATH_TO_FIT = Path("data/q5/vae_fit")
file = "M2_q5_p100_1.csv"
data = preprocessing(PATH_TO_DATA/file)
vae, intermediate_model = vae_fit(data,  K=5, known_variance=1., latent_dim=num_latent)

Epoch 1/1000
7/7 [==============================] - 8s 6ms/step - total_loss: 313.7945 - reconstruction_loss: 311.9833 - kl_loss: 1.8112
Epoch 2/1000
7/7 [==============================] - 0s 7ms/step - total_loss: 308.9655 - reconstruction_loss: 306.5159 - kl_loss: 2.4496
Epoch 3/1000
7/7 [==============================] - 0s 7ms/step - total_loss: 296.5862 - reconstruction_loss: 293.1691 - kl_loss: 3.4170
Epoch 4/1000
7/7 [==============================] - 0s 8ms/step - total_loss: 291.6923 - reconstruction_loss: 288.1252 - kl_loss: 3.5671
Epoch 5/1000
7/7 [==============================] - 0s 8ms/step - total_loss: 283.5858 - reconstruction_loss: 280.2915 - kl_loss: 3.2943
Epoch 6/1000
7/7 [==============================] - 0s 7ms/step - total_loss: 280.2996 - reconstruction_loss: 277.3037 - kl_loss: 2.9958
Epoch 7/1000
7/7 [==============================] - 0s 7ms/step - total_loss: 276.1759 - reconstruction_loss: 273.1747 - kl_loss: 3.0012
Epoch 8/1000
7/7 [=======================

In [23]:
vae.compile(optimizer="SGD")

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [13]:
save_output(data, vae, intermediate_model, file, PATH_TO_FIT)

7/7 [==============================] - 1s 12ms/step
